## Giới thiệu

Transfromer trong những năm gần đây đã làm thay đổi các chúng ta tiếp cận và giải quyết các bài toán thị giác máy tính như (Vision Transformer, Swin Transfromer, ...). Bên cạnh đó các kiến trúc tập trung vào attention ở trên thì các kiến trúc không sử dụng cơ chế attention như FNet, gMLP (gated Multilayer Perceptron) nổi lên như một phương pháp đầy tiềm năng, tận dụng sức mạnh và sự đơn giản của các lớp MLP thay vì phụ thuộc vào chú ý. gMLP hoạt động dựa trên cơ chế Spatial Gating Unit (SGU), một thành phần tinh gọn nhưng hiệu quả trong việc học các mối quan hệ không gian giữa các token mà không cần tới self-attention. Tuy nhiên, các hạn chế của gMLP trong việc mở rộng khả năng biểu diễn và xử lý các đặc điểm phức tạp vẫn là một rào cản lớn khi so sánh với các mô hình tiên tiến như Swin Transformer hoặc các kiến trúc hiện đại dựa trên CNN.
Để khắc phục những hạn chế này, nghiên cứu đề xuất một phiên bản cải tiến của gMLP, sử dụng các đặc trưng đa thức (trong nghiên cứu này sẽ sử dụng bậc 2) gọi là gMPLP.


---


## Ý tưởng
- Cho một input batch $ x \in \mathbb{R}^{N \times H \times W \times C} $. Nghiên cứu sử dụng $ (H, W) = (256, 256) $.
- Chia các hình ảnh đầu vào thành các ***patches*** và reshape. Với ***patch size = 16*** $\Rightarrow$ thu được $z$ có shape: $ (N, number\_of\_patches, patch\_height \times patch\_width \times C) $.

Nghiên cứu sử dụng một layer thực hiện các hoạt động sau:

- Chuẩn hóa: $ \overline{z} \leftarrow LN(z) $ \\
trong đó: $ LN $ là Layer Normalization.
- Tạo đặc trưng đa thức: $ p = [z, z^2, z^3,..., z^d] $. Với $d$ là bậc của đa thức.
- Áp dụng một phép chiếu tuyến tính với $p$:
$$ p \leftarrow p \cdot W + b $$
với $W$ là ma trận trọng số và $b$ là bias vector.
- Chuẩn hóa lần 2: $ p \leftarrow LN(p) $
- Áp dụng activation: $ output = activation(p) $. Ở đây activation có thể là $gelu$ hoặc $linear$.

---

## Xây dựng gMPLP block từ layer trên:

![gMPLP](MPLP_image.jpg)

## Dữ liệu:


- [https://ieee-dataport.org/documents/brain-tumor-mri-dataset](https://ieee-dataport.org/documents/brain-tumor-mri-dataset)

## Training Details

- Số lượng ảnh dùng để huấn luyện:
    - training: 5712
    - validation: 571
    - testing: 1311

- Huấn luyện với 25 epochs. 15 epochs đầu tiên $learning\_rate = 1e-3$. 10 epochs sau $learning\_rate = 1e-4$
- L4 GPU

- Huấn luyện và so sánh gMLP và gMPLP. mô hình gMLP sẽ được mở rộng theo chiều rộng (số units trong phép chiếu tuyến tính) và chiều sâu (số gMLP blocks) để có số lượng tham số tương đương với gMPLP.
    - Số chiều nhúng (embedding_dim) của gMLP: 320, số blocks: 4.
    - Số chiều nhúng (embedding_dim) của gMPLP: 256, số blocks: 2.

- Huấn luyện và so sánh ViT và ViT + gMPLP. Thay thế khối MLP bình thường trong ViT cho thành khối gMPLP cho model ViT + gMPLP.
    - Số chiều nhúng (embedding_dim) của ViT: 160, số Transformer blocks: 2.
    - Số chiều nhúng (embedding_dim) của ViT + gMPLP: 160, số Transfromer blocks: 1.



## Kết quả

### gMLP

![gMLP](gMLP.png)

### gMPLP

![gMPLP](gMPLP.png)

### ViT

![ViT](ViT.png)

### ViT + gMPLP

![ViT + gMPLP](ViT_gMPLP.png)

### Bảng so sánh

![compare_tb](compare_tb.png)